<a href="https://colab.research.google.com/github/n-jagadeep/PythonPractice/blob/main/ColabTime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-datastore 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
google-cloud-firestore 2.16.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!

In [3]:
%%timeit
from ortools.sat.python import cp_model

model = cp_model.CpModel()

horizon = 24

jobs = [
    {"duration": 1},
    {"fixed_start": 8 , "duration": 1},
    {"fixed_start": 6 , "duration": 1},
    {"preferred_start": 6 , "duration": 1},
    {"preferred_start": 7 , "duration": 1}
]
job_variables = {}
penalty_terms = []


for i, job in enumerate(jobs):
  if 'fixed_start' in job:
    start_time = model.NewIntVar(job['fixed_start'],job['fixed_start'],f"job{i}_start")
    #print(job['fixed_start'])
  elif 'preferred_start' in job:
    start_time = model.NewIntVar(0,horizon - job['duration'], f"job{i}_start")
    preferred_start = job['preferred_start']
    deviation = model.NewIntVar(0, horizon, f"deviation_job{i}")
    model.AddAbsEquality(deviation, start_time - preferred_start)
    penalty_terms.append(deviation)
    print(penalty_terms,deviation)
  else:
    start_time = model.NewIntVar(0,horizon - job['duration'], f"job{i}_start")
  job_variables[i] = {
    "start_time": start_time,
    "duration" : job['duration']
    }

intervals = []
for i, job in enumerate(jobs):
    start_time = job_variables[i]['start_time']
    duration = job_variables[i]['duration']
    end_time = start_time + duration
    intervals.append(model.NewIntervalVar(start_time, duration, end_time, f"Job{i}_interval"))

model.AddNoOverlap(intervals)

makespan = model.NewIntVar(0 , horizon , 'makespan')

job_end_times = [job_variables[i]['start_time'] + job_variables[i]['duration'] for i in range(len(jobs))]

model.AddMaxEquality(makespan, job_end_times)

model.Minimize(makespan + sum(penalty_terms))

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"schedule found {solver.ObjectiveValue()} hours")
    for i, job in enumerate(jobs):
        print(f"Job {i + 1} starts at {solver.Value(job_variables[i]['start_time'])} ")
else:
    print("No solution found.")





[deviation_job3(0..24)] deviation_job3
[deviation_job3(0..24), deviation_job4(0..24)] deviation_job4
schedule found 10.0 hours
Job 1 starts at 0 
Job 2 starts at 8 
Job 3 starts at 6 
Job 4 starts at 5 
Job 5 starts at 7 
[deviation_job3(0..24)] deviation_job3
[deviation_job3(0..24), deviation_job4(0..24)] deviation_job4
schedule found 10.0 hours
Job 1 starts at 0 
Job 2 starts at 8 
Job 3 starts at 6 
Job 4 starts at 5 
Job 5 starts at 7 
[deviation_job3(0..24)] deviation_job3
[deviation_job3(0..24), deviation_job4(0..24)] deviation_job4
schedule found 10.0 hours
Job 1 starts at 0 
Job 2 starts at 8 
Job 3 starts at 6 
Job 4 starts at 5 
Job 5 starts at 7 
[deviation_job3(0..24)] deviation_job3
[deviation_job3(0..24), deviation_job4(0..24)] deviation_job4
schedule found 10.0 hours
Job 1 starts at 0 
Job 2 starts at 8 
Job 3 starts at 6 
Job 4 starts at 5 
Job 5 starts at 7 
[deviation_job3(0..24)] deviation_job3
[deviation_job3(0..24), deviation_job4(0..24)] deviation_job4
schedule fo